In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd

In [2]:
CLIENT_ID = "cf48ede19908400693b2a5800a50a785"
CLIENT_SECRET = "c215b8a1d2e349768159f7a037669545"

In [3]:
token = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

<ipython-input-3-535d6a9da87f>:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  cache_token = token.get_access_token()


In [4]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","acousticness",
                              "energy","key","loudness","mode", "speechiness","instrumentalness","liveness",
                              "valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)
    
    while True:
        for track in playlist["items"]:
            # Create empty dict
            playlist_features = {}
            # Get metadata
            playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
            playlist_features["album"] = track["track"]["album"]["name"]
            playlist_features["track_name"] = track["track"]["name"]
            playlist_features["track_id"] = track["track"]["id"]
            
            # Get audio features
            audio_features = sp.audio_features(playlist_features["track_id"])[0]
            for feature in playlist_features_list[4:]:
                playlist_features[feature] = audio_features[feature]
        
            # Concat the dfs
            track_df = pd.DataFrame(playlist_features, index = [0])
            playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
            
        if playlist['next']:
            playlist = sp.next(playlist)
            print("new page")
            
        else:
            break
        
    return playlist_df

In [5]:
tracks = analyze_playlist("spotify","41JwIWrvh6vmoR76kkroO7")

new page
new page
new page
new page
new page


In [6]:
tracks = tracks.reset_index()
tracks['index'] += 1
tracks

,index,artist,album,track_name,track_id,danceability,acousticness,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1,HVOB,Rocco,2nd World,6U6TeOpp28DZPbv1t4pl1b,0.572,0.25500,0.544,7,-11.026,0,0.0274,0.818000,0.1040,0.0393,115.994,337536,4
1,2,Khalid,American Teen,8TEEN,5bgwqaRSS3M8WHWruHgSL5,0.830,0.22500,0.585,1,-5.802,1,0.0366,0.002090,0.0576,0.6420,105.030,228600,4
2,3,Damien Rice,9,9 Crimes,5GZEeowhvSieFDiR8fQ2im,0.346,0.91300,0.139,0,-15.326,1,0.0321,0.000077,0.0934,0.1160,136.168,217947,4
3,4,Radiohead,In Rainbows,15 Step,6dsq7Nt5mIFzvm5kIYNORy,0.600,0.04890,0.853,1,-7.584,1,0.0423,0.346000,0.0934,0.8470,94.457,237293,4
4,5,Coldplay,Viva La Vida (Prospekt's March Edition),42,1X2v8JtRJhGQEy1QIhZnl7,0.309,0.22100,0.636,5,-7.202,0,0.0344,0.002810,0.1020,0.1770,141.382,237400,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,563,Hozier,Hozier,Take Me To Church,3dYD57lRAUcMHufyqn9GcI,0.566,0.63400,0.664,4,-5.303,0,0.0464,0.000000,0.1160,0.4370,128.945,241688,4
563,564,Petit Biscuit,Night Trouble,Night Trouble,5LLtNp8qc3NaCIieZ2W4Dk,0.411,0.95100,0.364,1,-15.335,1,0.0743,0.930000,0.1260,0.0685,95.967,391257,3
564,565,The Lumineers,Cleopatra,Sleep On The Floor,44uOreAESeRIBaxkuCW4wN,0.389,0.24900,0.431,7,-8.061,1,0.0344,0.000000,0.1300,0.2750,142.140,211851,4
565,566,Brothers of Metal,Yggdrasil,Yggdrasil,2gGveBaLJQMtJ43X4UL5kH,0.469,0.00222,0.743,1,-5.570,0,0.0272,0.000111,0.2760,0.4810,86.953,272292,4


In [7]:
tracks.to_csv('new_tracks1.csv', index=False)